In [11]:
import pandas as pd

In [12]:
# !python -m pip install xlrd

In [33]:
jun_delim = pd.read_csv('./expenses/jun_2023_delimited.txt', sep=r'\s,', engine='python')
jun_delim.head(10)

,Date,Narration,"Value Dat,Debit Amount",Credit Amount,Chq/Ref Number,Closing Balance
26/05/23,NEFT CR-HSBC0400002-SHELL INDIA MARKETS PRIVAT...,26/05/23,0.0,93625.63,HSBCN23146678253,96217.59
27/05/23,UPI-PRADEEP SAKTHI S-PRADEEP213@OKICICI-ICIC00...,27/05/23,220.0,0.00,0000314763098948,95997.59
27/05/23,UPI-PRADEEP SAKTHI S-PRADEEP213@OKICICI-ICIC00...,27/05/23,500.0,0.00,0000314763111857,95497.59
27/05/23,UPI-ARAVIND AMSASEKAR A-ARAVIND.AAAA@OKICICI-I...,27/05/23,233.0,0.00,0000314763126188,95264.59
27/05/23,UPI-SOMESH KIRTHIK S P-SPSOMESH2001-2@OKHDFCBA...,27/05/23,5000.0,0.00,0000314763149696,90264.59
27/05/23,UPI-SOMESH KIRTHIK S P-SPSOMESH2001-2@OKHDFCBA...,27/05/23,6750.0,0.00,0000314763179200,83514.59
27/05/23,UPI-HILLTOP INN-8124600010@OKBIZICICI-ICIC0DC0...,27/05/23,1790.0,0.00,0000314763789230,81724.59
27/05/23,UPI-NAGENDRA BABU VANKA-BOBBY8158@YBL-SBIN001...,27/05/23,40.0,0.00,0000314768999964,81684.59
27/05/23,UPI-ARCHANA PURI-ARCHIEPURI@OKICICI-ICIC000603...,27/05/23,120.0,0.00,0000314770488463,81564.59
29/05/23,ACH D- BDACH-SBISMSMFB-TJMI5089064 ...,29/05/23,3000.0,0.00,0000001920500214,78564.59


In [63]:
# jun_xls = pd.read_excel('./expenses/jun_2023.xls')
# jun_xls
import openpyxl

sheet = openpyxl.load_workbook("./expenses/jun_2023.xls")

sheet

InvalidFileException: openpyxl does not support the old .xls file format, please use xlrd to read this file, or convert it to the more recent .xlsx file format.

In [146]:
import xlrd

book = xlrd.open_workbook("./expenses/jun_2023.xls")
sheet = book.sheet_by_index(0)

transaction_entries_start_idx: int = -1
transaction_entries_end_idx: int = -1
label_row_idx: int = -1

inside_transaction_section: bool = False

for idx, row in enumerate(sheet.get_rows()):
    # row is a list with xlrd.sheet.Cell objects
    first_cell: xlrd.sheet.Cell = row[0]
    
    # ignore lines that dont start with asterisk, 
    # asterisk lines are markers for the transaction rows starting and ending
    if not first_cell.value.startswith('*'):
        continue
       
    # check if this is start of list of transactions
    row_values          = sheet.row_values(idx)
    filtered_row_values = list(filter(lambda cell: cell != '', row_values))

    # start and end of the transaction list has *** in each cell
    # after filtering out empty cells, if the length of the list is same as original, 
    # it means all cells have text (and these will be asterisks according to the format)
    # store these row numbers
    if len(row_values) == len(filtered_row_values):
        if transaction_entries_start_idx == - 1:
            transaction_entries_start_idx = idx + 1
            label_row_idx = idx - 1
            
        else:
            # there's a blank line before the asterisk line
            transaction_entries_end_idx = idx - 2
            break

    
labels = sheet.row_values(label_row_idx)

print(labels, transaction_entries_start_idx + 1, transaction_entries_end_idx + 1)

['Date', 'Narration', 'Chq./Ref.No.', 'Value Dt', 'Withdrawal Amt.', 'Deposit Amt.', 'Closing Balance'] 23 110


In [149]:
from dataclasses import dataclass
import datetime

@dataclass
class Transaction:
    date: datetime.date
    narration: str
    reference_id: str
    value_date: datetime.date
    withdraw_amount: float
    deposit_amount: float
    closing_balance: float    

In [148]:
for idx, row in enumerate(sheet.get_rows()):
    if transaction_entries_start_idx <= idx <= transaction_entries_end_idx:
        print(sheet.row_values(idx))
        


['26/05/23', 'NEFT CR-HSBC0400002-SHELL INDIA MARKETS PRIVATE LIMITED-SIBI AKKASH SUNDAR-HSBCN23146678253', 'HSBCN23146678253', '26/05/23', '', 93625.63, 96217.59]
['27/05/23', 'UPI-PRADEEP SAKTHI S-PRADEEP213@OKICICI-ICIC0002181-314763098948-HSR HIGH STREET', '0000314763098948', '27/05/23', 220.0, '', 95997.59]
['27/05/23', 'UPI-PRADEEP SAKTHI S-PRADEEP213@OKICICI-ICIC0002181-314763111857-AROMAS OF BIRIYANI', '0000314763111857', '27/05/23', 500.0, '', 95497.59]
['27/05/23', 'UPI-ARAVIND AMSASEKAR A-ARAVIND.AAAA@OKICICI-ICIC0002181-314763126188-KODAI CAMP GROUND', '0000314763126188', '27/05/23', 233.0, '', 95264.59]
['27/05/23', 'UPI-SOMESH KIRTHIK S P-SPSOMESH2001-2@OKHDFCBANK-HDFC0003734-314763149696-UPI', '0000314763149696', '27/05/23', 5000.0, '', 90264.59]
['27/05/23', 'UPI-SOMESH KIRTHIK S P-SPSOMESH2001-2@OKHDFCBANK-HDFC0003734-314763179200-APRIL RENT', '0000314763179200', '27/05/23', 6750.0, '', 83514.59]
['27/05/23', 'UPI-HILLTOP INN-8124600010@OKBIZICICI-ICIC0DC0099-314763789

In [51]:
import re

with open("./expenses/jun_2023_delimited.txt") as f:
    lines = f.readlines()[1:] # first link is blank, skip it
    
    label_line, rest_lines = lines[0], lines[1:]
    
    labels = re.split(string=label_line.strip(), pattern= r'\s*,')
    print(labels)
    
    print(*rest_lines[:3], sep='\n')
    print("********************************************")
    for data in rest_lines:
        parts = re.split(string=data, pattern=r'\s*,')
        parts_cleaned = list(map(lambda part: part.strip(), parts))
        print(parts_cleaned)

['Date', 'Narration', 'Value Dat', 'Debit Amount', 'Credit Amount', 'Chq/Ref Number', 'Closing Balance']
 26/05/23  ,NEFT CR-HSBC0400002-SHELL INDIA MARKETS PRIVATE LIMITED-SIBI AKKASH SUNDAR-HSBCN23146678253                              ,26/05/23 ,          0.00     ,      93625.63     ,HSBCN23146678253       ,     96217.59  

 27/05/23  ,UPI-PRADEEP SAKTHI S-PRADEEP213@OKICICI-ICIC0002181-314763098948-HSR HIGH STREET                                         ,27/05/23 ,        220.00     ,          0.00     ,0000314763098948       ,     95997.59  

 27/05/23  ,UPI-PRADEEP SAKTHI S-PRADEEP213@OKICICI-ICIC0002181-314763111857-AROMAS OF BIRIYANI                                      ,27/05/23 ,        500.00     ,          0.00     ,0000314763111857       ,     95497.59  

********************************************
['26/05/23', 'NEFT CR-HSBC0400002-SHELL INDIA MARKETS PRIVATE LIMITED-SIBI AKKASH SUNDAR-HSBCN23146678253', '26/05/23', '0.00', '93625.63', 'HSBCN23146678253', '96217.59']
['2